# purpose

- fetch last 100 github issues
- try various ways to filter & search through them


# structure

- **fetch issues**
  - fetch the last 10 open and closed issues
  - that allows me to close open issues or reopen closed issues
- **dataframe data**
  - make a pandas dataframe from the data fetched from github
  - this is a nested dictionary - each comment and lable is a node inside the corresponding issue
  - so the comments and labels have been exploded into their own separate lines
- **search dataframe**
  - try scalar value search
  - try filtering with multiple parameters
  - use query api


# setup


In [ ]:
# fetch github token, python packages, queries and other parameters

%run ../../100-set_parameters/100-set_parameters.ipynb # magic commands

# fetch issues


In [ ]:
# query parameters
# specify the query parameters to be used while fetching issues
# github does not allow fetching more than 100 issues

fetch_issue_parameters = {
    "repository_name": "tensorflow",
    "owner_name": "tensorflow",
    "number_of_issues": 100
  }

In [ ]:
# store result

last_few_issues = global_query(fetch_recent_issues, fetch_issue_parameters)

## dictionary to dataframe


In [ ]:
# drill down
# take the dictionary result from previous step
# reach out to a particular level within that dictionary (edges in this case)
# store that inside a dataframe

df = pd.DataFrame(d['node'] for d in last_few_issues['data']['repository']['issues']['edges'])

## flatten nested data

- GitHub responds with a nested structure for the query above.
- in this response, each issue may contain various sub-elements, such as labels and comments
- for example, an issue might have the following structure:

```json
{
	"id": "issue-id",
	"title": "Example Issue",
	"labels": [{ "name": "bug" }, { "name": "enhancement" }],
	"comments": [{ "body": "This is a comment." }, { "body": "Another comment." }]
}
```

- in this nested structure, labels and comments are each grouped under a single issue, but for visualizations, we need to flatten this structure or explode it
- if an issue has multiple labels or comments, we want each label or comment to be represented as a separate record associated with the issue
- to achieve this, we can use the following code to flatten the nested data into a DataFrame, enabling easy visualization and analysis:


## comments


In [ ]:
# access comments

df['comments'] = df['comments'].str['edges']

In [ ]:
# flatten comments

# the comments for each issue are nested under the 'comments' key
# code below extracts the nested comments and flatten them into a simpler structure

df_flat_comments = df.explode('comments')
df_flat_comments['comments'] = df_flat_comments['comments'].str['node'].str['body']

## labels


In [ ]:
# access labels
df_flat_comments['labels'] = df_flat_comments['labels'].str['edges']

In [ ]:
# flatten labels

df_flat_comments_and_labels = df_flat_comments.explode('labels')
df_flat_comments_and_labels['labels'] = df_flat_comments_and_labels['labels'].str['node'].str['name']

## final dataframe


In [ ]:
# print to screen

df=df_flat_comments_and_labels
df

# search dataframe

- the previous steps covered
  - fetching data from graphql api server
  - putting the response inside a dataframe
  - flattening it out
- the next steps involve
  - searching through the dataframe
  - i will also use the DataFrames Query API


## searching syntax

- the pandas equivalent to sql query
  - `select * from table where column_name = some_value`
- is
  - for single condition
    - `table[table.column_name == some_value]`
  - for multiple conditions
    - `table[(table.column_name == some_value) | (table.column_name2 == some_value2)]`
- check [stackoverflow](https://stackoverflow.com/a/31296878) for more details


## scalar-value search

- to select rows whose column value equals a scalar, some_value, use `==`
- in the next 2 sections have examples for
  - search for an integer
  - search for a string
- check [stackoverflow](https://stackoverflow.com/a/17071908) for more details


In [ ]:
# scalar-search integer

df.loc[df['number'] > 13].head(3) # github issue number
#note that github fetches only 100 records at a time

In [ ]:
# scalar-search string

df.loc[df['labels'] == "type:bug"].head(3)

## iterable

- search for a list of values within a particular column
- [link](https://stackoverflow.com/a/17071908)


In [ ]:
# code

df.loc[df['labels'].isin(["type:bug", "type:build/install"])].head(3)

## multiple conditions

- notice how 2 conditions have been used for searching through the dataframe
- one condition involves a search string & the other has an integer
- [link](https://stackoverflow.com/a/17071908)


In [ ]:
# code

df.loc[(df['labels'] == "type:build/install") & (df['labels'] == "subtype: ubuntu/linux")]#.head(3)

# negtation

- search for rows whose value DOES NOTE equal to the search input
- 2 examples given below
  - search for scalar values
  - search for iterable values
- [link](https://stackoverflow.com/a/17071908)


In [ ]:
# scalar

df.loc[df['labels'] != "type:bug"].head(3)

In [ ]:
# iterable

df.loc[~df['labels'].isin(["type:bug", "type:build/install"])].head(3)

## query api

- `DataFrame.query()` is a method of pandas
- it is used to query the rows based on the expression (single or multiple column conditions)
- in case i want to update the previously created DataFrame `df`, then i use `inplace=True` argument for the method
- links
  - [sparkbyexamples-df_query_api](https://sparkbyexamples.com/pandas/pandas-dataframe-query-examples/)
  - [geeksforgeeks-df_query_api](https://www.geeksforgeeks.org/python-filtering-data-with-pandas-query-method/)


### basic example


In [ ]:
# code

df.query('labels == "type:build/install"').head(3)

### syntax

- `DataFrame.query(expr, inplace=False, **kwargs)`
- where
  - `expr`
    - expression takes conditions to query rows
  - `inplace`
    - defaults to false
    - when set to true, it updates the referring DataFrame
    - the query() method returns none
  - `**kwargs` – Keyword arguments that works with eval()


In [ ]:
# calculate yesterday's date
yesterday = datetime.today().date() - timedelta(days=1)
df['createdOnDate'] = pd.to_datetime(df['createdAt']).dt.date #you can modify this to search for issues created today

# use inplace
df_inplace = df.copy() # https://www.w3schools.com/python/pandas/ref_df_copy.asp
df_inplace.query("createdOnDate == @yesterday", inplace=True)
df_inplace

In [ ]:
# list

search_list = ['type:bug', 'type:build/install']
df.query("labels in @search_list").head(3)